Installing Required Packages

In [0]:
!pip install python-docx
!pip install nltk
!pip install keras
!pip install gensim
!pip install tensorflow
!pip install sklearn

Reading Data From S3 Buckets

In [0]:

import pandas as pd
import boto3

bucket = "projectworks"
file_name = "Cancer.csv"

s3 = boto3.client('s3') 

obj = s3.get_object(Bucket= bucket, Key= file_name) 


initial_df = pd.read_csv(obj['Body'])

data=initial_df["Description"]

In [0]:
voca=[]
text=''
temp_text=''
ct=0
l = list(data)
for i in l:
    text = text+""+i
    voca.append(i)   
    

In [0]:
def find_y(voca):
    c=len(voca)
    y=[1 for i in range(0,c)]
    for i in range(0,c) :
        Word=nltk.word_tokenize(voca[i])
        if 'II' in Word:
            y[i]=2
        if 'III' in Word:
            y[i]=3
        if 'IV' in Word:
            y[i]=4
        if 'V' in Word:
            y[i]=5
        if 'VI' in Word:
            y[i]=6    
        
    return y

In [0]:
def bag_of_words(doc_words,doc_temp,vocabulary):    # boolean representation 
    bool_vec = []
    temp_vec=[]
      
    for i in range(len(doc_words)):
        for j in range(len(vocabulary)):
            if vocabulary[j] in doc_words[i]:
               temp_vec.append(1)
            else:
                temp_vec.append(0)      
        
        bool_vec.append(temp_vec)       
        temp_vec=[]  
        
    return bool_vec

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def clean_text(text):
    w_text = nltk.word_tokenize(text)
    
    punc_text=[]
    for i in w_text:
        if i not in string.punctuation:
            punc_text.append(i) 

    for i in punc_text:
        i=i.lower()
    
    stop_words=set(stopwords.words('english'))

    non_stop_text=[]        
    for i in punc_text:
        if i not in stop_words:
            non_stop_text.append(i)
    
    unique_text = list(set(non_stop_text))

    return unique_text


In [0]:
def doc_words_cleaning(voca):       # cleans and preprocesses each document 
    doc_words = []    
    for i in range(len(voca)):
        doc_words.append(clean_text(voca[i]))
    return doc_words


In [0]:
def bag_of_words(doc_words,doc_temp,vocabulary):    # boolean representation 
    bool_vec = []
    temp_vec=[]
      
    for i in range(len(doc_words)):
        for j in range(len(vocabulary)):
            if vocabulary[j] in doc_words[i]:
               temp_vec.append(1)
            else:
                temp_vec.append(0)      
        
        bool_vec.append(temp_vec)       
        temp_vec=[]  
        
    return bool_vec

In [0]:
def doc_term_freq(boo):
    doc_freq=[]
    for i in range(len(boo)):
        summ=0
        for j in range(len(boo[0])):
            if boo[i][j] == 1:
                summ=summ+1
        doc_freq.append(summ)
    return doc_freq







In [0]:
# count of no of vocabulary words in each document

def count_voca_words(bove):
    a=[]
    for i in range(len(bove)):
        a.append(bove[i].count(1))    
    return a


In [0]:
# term frequency [NAIVE] :-  

def term_frequency_naive(doc_words,vocabulary):   
    tf=[]
    temp=[]
    for i in range(len(doc_words)):
        for j in vocabulary:
            temp.append(doc_words[i].count(j))
        tf.append(list(zip(temp,vocabulary)))
        temp=[]
    
    return tf            






In [0]:
# tf - idf term frequency 

def tf(doc_words,vocabulary):
    tf=[]
    temp=[]
    for i in range(len(doc_words)):
        for j in vocabulary:
            temp.append((doc_words[i].count(j))/len(doc_words[i]))
        tf.append(temp)
        temp=[]
    return tf




In [0]:
def idf(doc_words,vocabulary):
    import math
    doc_count = 0
    idf=[]
    for i in vocabulary:
        for j in range(len(doc_words)):
            if i in doc_words[j]:
                doc_count = doc_count + 1
        idf.append(math.log(len(doc_words)/(doc_count)))
        doc_count = 0 
    
    return idf





In [0]:
def tf_idf_score_calc(tf,idf):
    tf_idf = []
    temp=[]
    for i in range(len(tf)):
        for j in range(len(idf)):
            if tf[i][j] !=0 and idf[j] !=0:
                temp.append(tf[i][j]*idf[j])
            else:
                temp.append(0)
        tf_idf.append(temp)
        temp=[]
    
    return tf_idf



In [0]:
vocabulary = clean_text(text)

doc_words = doc_words_cleaning(voca)

bow = bag_of_words(doc_words,voca,vocabulary)

y=find_y(voca)


doc_freq = count_voca_words(bow)   

term_freq_naive = term_frequency_naive(doc_words,vocabulary)

term_frequency = tf(doc_words,vocabulary)

inverse_doc_freq = idf(doc_words,vocabulary)

tf_idf_score = tf_idf_score_calc(term_frequency,inverse_doc_freq)



Building Neural Network Model

In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

import tensorflow as tfl
#Parameters
L_rate=0.1
epochs=100000000

n_input=784
n_hidden=200
n_output=6

X=tfl.placeholder("float",[None,n_input])
Y=tfl.placeholder("float",[None,n_output])

w1=tfl.Variable(tfl.random_normal([n_input,n_hidden]))
w2=tfl.Variable(tfl.random_normal([n_hidden,n_output]))

bias1=tfl.Variable(tfl.random_normal([n_hidden]))
bias2=tfl.Variable(tfl.random_normal([n_output]))

def model(X):
  layer1=tfl.add(tfl.matmul(X,w1),bias1)
  layer1=tfl.nn.relu(layer1)
  output_layer=tfl.matmul(layer1,w2)+bias2
  return output_layer
  




Bilding The model

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bow, dummy_y)
#X_train, X_test, y_train, y_test = train_test_split(term_frequency, dummy_y)
#X_train, X_test, y_train, y_test = train_test_split(tf_idf_score, dummy_y)

pred=model(X)
cost=tfl.reduce_mean(tfl.nn.softmax_cross_entropy_with_logits(logits=pred,labels=Y))
optimizer =tfl.train.AdamOptimizer(L_rate).minimize(cost)
init=tfl.global_variables_initializer()

with tfl.Session() as sess:
  sess.run(init)
  for epochs in range(epochs):
    _.c=sess.run([optimizer,cost],feed_dict={X:X_train,Y:y_train})
    
  test_result = sess.run(pred,feed_dict= {X:X_train})
  y_pred=tfl.equal(tfl.argmax(test_result,1),tfl.argmax(y_train,1))
  accuracy=tfl.reduce_mean(tfl.cast(y_pred,"float"))
  
  print("Accuracy:",accuracy.eval({X:X_test,Y:y_test}))

In [0]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(bow, dummy_y)
X_train, X_test, y_train, y_test = train_test_split(term_frequency, dummy_y)
#X_train, X_test, y_train, y_test = train_test_split(tf_idf_score, dummy_y)

pred=model(X)
cost=tfl.reduce_mean(tfl.nn.softmax_cross_entropy_with_logits(logits=pred,labels=Y))
optimizer =tfl.train.AdamOptimizer(L_rate).minimize(cost)
init=tfl.global_variables_initializer()

with tfl.Session() as sess:
  sess.run(init)
  for epochs in range(epochs):
    _.c=sess.run([optimizer,cost],feed_dict={X:X_train,Y:y_train})
    
  test_result = sess.run(pred,feed_dict= {X:X_train})
  y_pred=tfl.equal(tfl.argmax(test_result,1),tfl.argmax(y_train,1))
  accuracy=tfl.reduce_mean(tfl.cast(y_pred,"float"))
  
  print("Accuracy:",accuracy.eval({X:X_test,Y:y_test}))

In [0]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(bow, dummy_y)
#X_train, X_test, y_train, y_test = train_test_split(term_frequency, dummy_y)
X_train, X_test, y_train, y_test = train_test_split(tf_idf_score, dummy_y)

pred=model(X)
cost=tfl.reduce_mean(tfl.nn.softmax_cross_entropy_with_logits(logits=pred,labels=Y))
optimizer =tfl.train.AdamOptimizer(L_rate).minimize(cost)
init=tfl.global_variables_initializer()

with tfl.Session() as sess:
  sess.run(init)
  for epochs in range(epochs):
    _.c=sess.run([optimizer,cost],feed_dict={X:X_train,Y:y_train})
    
  test_result = sess.run(pred,feed_dict= {X:X_train})
  y_pred=tfl.equal(tfl.argmax(test_result,1),tfl.argmax(y_train,1))
  accuracy=tfl.reduce_mean(tfl.cast(y_pred,"float"))
  
  print("Accuracy:",accuracy.eval({X:X_test,Y:y_test}))

Accuracy: 0.28
